# Installing retinanet library

In [ ]:
# a = []
# while(1):
#     a.append(1)

In [ ]:
!git clone --recursive https://github.com/fizyr/keras-retinanet.git
!ls
!cd keras-retinanet && pip install . --user
!pip install Cython
!pip install --user git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
!cd keras-retinanet/snapshots && \
wget https://github.com/fizyr/keras-retinanet/releases/download/0.2/resnet50_coco_best_v2.0.3.h5

!pip install --upgrade git+https://github.com/fizyr/keras-retinanet
!pip install --upgrade git+https://github.com/broadinstitute/keras-resnet

import keras
import keras_resnet

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5762 (delta 0), reused 1 (delta 0), pack-reused 5757
Receiving objects: 100% (5762/5762), 13.38 MiB | 32.39 MiB/s, done.
Resolving deltas: 100% (3865/3865), done.
Submodule 'tests/test-data' (https://github.com/fizyr/keras-retinanet-test-data.git) registered for path 'tests/test-data'
Cloning into '/content/keras-retinanet/tests/test-data'...
remote: Enumerating objects: 45, done.        
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45        
Submodule path 'tests/test-data': checked out '98404379fbf1ff1273d01db835c10cc83a4f8007'
drive  keras-retinanet	sample_data
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=169702 sha256=cbc7680823ab6d57839cb3c8ab30853278255dabc2fcdc06f5d5fa77ede5f54e
  Stored in director

Using TensorFlow backend.


In [ ]:

import xml.etree.ElementTree as ET
import os
import numpy as np
import keras
import math
import tensorflow as tf
import cv2
from os import listdir, walk
from os.path import join
from keras_retinanet.bin.train import create_generators,create_models,create_callbacks
from keras_retinanet.models import backbone,load_model,convert_model
from keras_retinanet.utils.config import read_config_file,parse_anchor_parameters
from keras_retinanet.utils.visualization import draw_boxes
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa


# Training Left Foot model

In [ ]:

# tf.set_random_seed(31) # SEEDS MAKE RESULTS MORE REPRODUCABLE
np.random.seed(17)
classes = np.arange(0, 6, 1).tolist()

import pandas as pd
tr_annots = pd.read_csv('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/all_foot_train_03_05.csv', header = None)
tr_annots['ids'] = tr_annots[0].apply(lambda x: x.split("/")[-1].split(".")[0])
train_ids = tr_annots['ids'].unique().tolist()

import pandas as pd
tr_annots = pd.read_csv('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/all_foot_val_03_06.csv', header = None)
tr_annots['ids'] = tr_annots[0].apply(lambda x: x.split("/")[-1].split(".")[0])
val_ids = tr_annots['ids'].unique().tolist()

print(len(train_ids), len(val_ids))

355 61


# Previous config files

In [ ]:
# # with open('config.ini','w') as f:
# #     f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512\nstrides = 8 16 32 64 128\nratios  = 1 1.5 2 2.5 \nscales  = 1 2\n')

# #Updating the anchor parameters
# with open('config.ini','w') as f:
#     f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512\nstrides = 8 16 32 64 128\nratios  = 1.5 1.8 2 2.2 2.5 2.8 3.0\nscales  = 1 1.2 1.6 2 3\n')


## New config parameters for anchors

In [ ]:
#Updating the anchor parameters
# with open('config.ini','w') as f:
#     f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512\nstrides = 8 16 32 64 128\nratios  = 0.8 1 1.3 1.5 1.8 2 2.2 2.5 2.8 3.0 3.2\nscales  = 1 1.2 1.6\n')

In [ ]:
# # New ratios parameters for training as previous didn't help
# with open('config.ini','w') as f:
#     f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512\nstrides = 8 16 32 64 128\nratios  = 0.8 1 1.3 1.5 1.8 2 2.2 2.5 2.8 3.0 3.2\nscales  = 1 1.2 1.6\n')


In [ ]:
# Updating the parameters
# New ratios parameters for training as previous didn't help
with open('config.ini','w') as f:
    f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512 1024\nstrides = 8 16 32 64 128 256\nratios  = 1 1.5 2 2.5 3\nscales  = 1 1.2 1.6\n')


In [ ]:

b = backbone('resnet50')

# Increasing batch size from 4 -> 8
class args:
    batch_size = 8
    config = read_config_file('config.ini')
    random_transform = True # Image augmentation
    annotations = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/all_foot_train_03_05.csv'
    val_annotations = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/all_foot_val_03_06.csv'
    classes = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/Foot class names.csv'
    image_min_side = 1000
    image_max_side = 1400
    no_resize = False
    dataset_type = 'csv'
    tensorboard_dir = ''
    evaluation = False
    snapshots = True
    snapshot_path = "/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2 continued"
    backbone = 'resnet50'
    epochs = 200
    steps = len(train_ids)//(batch_size)
    weighted_average = True
    reduce_lr_factor = 0.1
    reduce_lr_patience = 4
    compute_val_loss = True
    iou_threshold = 0.6
    nms_threshold = 0.5
    score_threshold = 0.15
    anchors = True
    resize = True
    display_name = 'Anchors'
    no_gui = False

In [ ]:
train_gen, valid_gen = create_generators(args, b.preprocess_image)

In [ ]:

from keras_retinanet.preprocessing.generator import Generator
# from ..utils.image import read_image_bgr
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.image import random_visual_effect_generator

import numpy as np
from PIL import Image
from six import raise_from

import csv
import sys
import os.path
from collections import OrderedDict

import PIL.Image
import PIL.ImageOps
import numpy as np
from os import makedirs 


def exif_transpose(img):
    if not img:
        return img

    exif_orientation_tag = 274

    # Check for EXIF data (only present on some files)
    if hasattr(img, "_getexif") and isinstance(img._getexif(), dict) and exif_orientation_tag in img._getexif():
        exif_data = img._getexif()
        orientation = exif_data[exif_orientation_tag]

        # Handle EXIF Orientation
        if orientation == 1:
            # Normal image - nothing to do!
            pass
        elif orientation == 2:
            # Mirrored left to right
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 3:
            # Rotated 180 degrees
            img = img.rotate(180)
        elif orientation == 4:
            # Mirrored top to bottom
            img = img.rotate(180).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 5:
            # Mirrored along top-left diagonal
            img = img.rotate(-90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 6:
            # Rotated 90 degrees
            img = img.rotate(-90, expand=True)
        elif orientation == 7:
            # Mirrored along top-right diagonal
            img = img.rotate(90, expand=True).transpose(PIL.Image.FLIP_LEFT_RIGHT)
        elif orientation == 8:
            # Rotated 270 degrees
            img = img.rotate(90, expand=True)

    return img


def load_image_file(file, mode='RGB'):
    # Load the image with PIL
    img = PIL.Image.open(file)

    if hasattr(PIL.ImageOps, 'exif_transpose'):
        # Very recent versions of PIL can do exit transpose internally
        img = PIL.ImageOps.exif_transpose(img)
    else:
        # Otherwise, do the exif transpose ourselves
        img = exif_transpose(img)

    img = img.convert(mode)

    return np.array(img)

def _parse(value, function, fmt):
    """
    Parse a string into a value, and format a nice ValueError if it fails.

    Returns `function(value)`.
    Any `ValueError` raised is catched and a new `ValueError` is raised
    with message `fmt.format(e)`, where `e` is the caught `ValueError`.
    """
    try:
        return function(value)
    except ValueError as e:
        raise_from(ValueError(fmt.format(e)), None)


def _read_classes(csv_reader):
    """ Parse the classes file given by csv_reader.
    """
    result = OrderedDict()
    for line, row in enumerate(csv_reader):
        line += 1

        try:
            class_name, class_id = row
        except ValueError:
            raise_from(ValueError('line {}: format should be \'class_name,class_id\''.format(line)), None)
        class_id = _parse(class_id, int, 'line {}: malformed class ID: {{}}'.format(line))

        if class_name in result:
            raise ValueError('line {}: duplicate class name: \'{}\''.format(line, class_name))
        result[class_name] = class_id
    return result


def _read_annotations(csv_reader, classes):
    """ Read annotations from the csv_reader.
    """
    result = OrderedDict()
    for line, row in enumerate(csv_reader):
        line += 1

        try:
            img_file, x1, y1, x2, y2, class_name = row[:6]
        except ValueError:
            raise_from(ValueError('line {}: format should be \'img_file,x1,y1,x2,y2,class_name\' or \'img_file,,,,,\''.format(line)), None)

        if img_file not in result:
            result[img_file] = []

        # If a row contains only an image path, it's an image without annotations.
        if (x1, y1, x2, y2, class_name) == ('', '', '', '', ''):
            continue

        x1 = _parse(x1, int, 'line {}: malformed x1: {{}}'.format(line))
        y1 = _parse(y1, int, 'line {}: malformed y1: {{}}'.format(line))
        x2 = _parse(x2, int, 'line {}: malformed x2: {{}}'.format(line))
        y2 = _parse(y2, int, 'line {}: malformed y2: {{}}'.format(line))

        # Check that the bounding box is valid.
        if x2 <= x1:
            raise ValueError('line {}: x2 ({}) must be higher than x1 ({})'.format(line, x2, x1))
        if y2 <= y1:
            raise ValueError('line {}: y2 ({}) must be higher than y1 ({})'.format(line, y2, y1))

        # check if the current class name is correctly present
        if class_name not in classes:
            raise ValueError('line {}: unknown class name: \'{}\' (classes: {})'.format(line, class_name, classes))

        result[img_file].append({'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2, 'class': class_name})
    return result


def _open_for_csv(path):
    """ Open a file with flags suitable for csv.reader.

    This is different for python2 it means with mode 'rb',
    for python3 this means 'r' with "universal newlines".
    """
    if sys.version_info[0] < 3:
        return open(path, 'rb')
    else:
        return open(path, 'r', newline='')


class CSVGenerator(Generator):
    """ Generate data for a custom CSV dataset.

    See https://github.com/fizyr/keras-retinanet#csv-datasets for more information.
    """

    def __init__(
        self,
        csv_data_file,
        csv_class_file,
        base_dir=None,
        **kwargs
    ):
        """ Initialize a CSV data generator.

        Args
            csv_data_file: Path to the CSV annotations file.
            csv_class_file: Path to the CSV classes file.
            base_dir: Directory w.r.t. where the files are to be searched (defaults to the directory containing the csv_data_file).
        """
        self.image_names = []
        self.image_data  = {}
        self.base_dir    = base_dir

        # Take base_dir from annotations file if not explicitly specified.
        if self.base_dir is None:
            self.base_dir = os.path.dirname(csv_data_file)

        # parse the provided class file
        try:
            with _open_for_csv(csv_class_file) as file:
                self.classes = _read_classes(csv.reader(file, delimiter=','))
        except ValueError as e:
            raise_from(ValueError('invalid CSV class file: {}: {}'.format(csv_class_file, e)), None)

        self.labels = {}
        for key, value in self.classes.items():
            self.labels[value] = key

        # csv with img_path, x1, y1, x2, y2, class_name
        try:
            with _open_for_csv(csv_data_file) as file:
                self.image_data = _read_annotations(csv.reader(file, delimiter=','), self.classes)
        except ValueError as e:
            raise_from(ValueError('invalid CSV annotations file: {}: {}'.format(csv_data_file, e)), None)
        self.image_names = list(self.image_data.keys())

        super(CSVGenerator, self).__init__(**kwargs)

    def size(self):
        """ Size of the dataset.
        """
        return len(self.image_names)

    def num_classes(self):
        """ Number of classes in the dataset.
        """
        return max(self.classes.values()) + 1

    def has_label(self, label):
        """ Return True if label is a known label.
        """
        return label in self.labels

    def has_name(self, name):
        """ Returns True if name is a known class.
        """
        return name in self.classes

    def name_to_label(self, name):
        """ Map name to label.
        """
        return self.classes[name]

    def label_to_name(self, label):
        """ Map label to name.
        """
        return self.labels[label]

    def image_path(self, image_index):
        """ Returns the image path for image_index.
        """
        return os.path.join(self.base_dir, self.image_names[image_index])

    def image_aspect_ratio(self, image_index):
        """ Compute the aspect ratio for an image with image_index.
        """
        # PIL is fast for metadata
        image = Image.open(self.image_path(image_index))
        return float(image.width) / float(image.height)

    def load_image(self, image_index):
        """ Load an image at the image_index.
        """
        return load_image_file(self.image_path(image_index))

    def load_annotations(self, image_index):
        """ Load annotations for an image_index.
        """
        path        = self.image_names[image_index]
        annotations = {'labels': np.empty((0,)), 'bboxes': np.empty((0, 4))}

        for idx, annot in enumerate(self.image_data[path]):
            annotations['labels'] = np.concatenate((annotations['labels'], [self.name_to_label(annot['class'])]))
            annotations['bboxes'] = np.concatenate((annotations['bboxes'], [[
                float(annot['x1']),
                float(annot['y1']),
                float(annot['x2']),
                float(annot['y2']),
            ]]))

        return annotations

def mAP(y_true, y_pred):
    num_classes = y_true.shape[1]
    average_precisions = []
    relevant = K.sum(K.round(K.clip(y_true, 0, 1)))
    tp_whole = K.round(K.clip(y_true * y_pred, 0, 1))
    for index in range(num_classes):
      temp = K.sum(tp_whole[:,:index+1],axis=1)
      average_precisions.append(temp * (1/(index + 1)))
    AP = Add()(average_precisions) / relevant
    mAP = K.mean(AP,axis=0)
    return mAP

def create_callbacks(model, training_model, prediction_model, validation_generator, args):
    """ Creates the callbacks to use during training.

    Args
        model: The base model.
        training_model: The model that is used for training.
        prediction_model: The model that should be used for validation.
        validation_generator: The generator for creating validation data.
        args: parseargs args object.

    Returns:
        A list of callbacks used for training.
    """
    callbacks = []

    tensorboard_callback = None

    if args.tensorboard_dir:
        makedirs(args.tensorboard_dir)
        tensorboard_callback = keras.callbacks.TensorBoard(
            log_dir                = args.tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = args.batch_size,
            write_graph            = True,
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
        )

    if args.evaluation and validation_generator:
        if args.dataset_type == 'coco':
            from ..callbacks.coco import CocoEval

            # use prediction model for evaluation
            evaluation = CocoEval(validation_generator, tensorboard=tensorboard_callback)
        else:
            evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback, weighted_average=args.weighted_average)
        evaluation = RedirectModel(evaluation, prediction_model)
        callbacks.append(evaluation)

    # save the model
    if args.snapshots:
        # ensure directory created first; otherwise h5py will error after epoch.
        makedirs(args.snapshot_path)
        checkpoint = keras.callbacks.ModelCheckpoint(
            os.path.join(
                args.snapshot_path,
                '{backbone}_{dataset_type}_{{epoch:02d}}.h5'.format(backbone=args.backbone, dataset_type=args.dataset_type)
            ),
            verbose=1,
            # save_best_only=True,
            # monitor="mAP",
            # mode='max'
        )
        checkpoint = RedirectModel(checkpoint, model)
        callbacks.append(checkpoint)

    callbacks.append(keras.callbacks.ReduceLROnPlateau(
        monitor    = 'loss',
        factor     = args.reduce_lr_factor,
        patience   = args.reduce_lr_patience,
        verbose    = 1,
        mode       = 'auto',
        min_delta  = 0.0001,
        cooldown   = 0,
        min_lr     = 0
    ))

    callbacks.append(keras.callbacks.EarlyStopping(
        monitor    = 'classification_loss',
        patience   = 20,
        mode       = 'max',
        min_delta  = 0.01
    ))

    if args.tensorboard_dir:
        callbacks.append(tensorboard_callback)

    return callbacks

def create_generators(args, preprocess_image):
    """ Create generators for training and validation.

    Args
        args             : parseargs object containing configuration for generators.
        preprocess_image : Function that preprocesses an image for the network.
    """
    common_args = {
        'batch_size'       : args.batch_size,
        'config'           : args.config,
        'image_min_side'   : args.image_min_side,
        'image_max_side'   : args.image_max_side,
        'no_resize'        : args.no_resize,
        'preprocess_image' : preprocess_image,
    }

    # create random transform generator for augmenting training data
    if args.random_transform:
        transform_generator = random_transform_generator(
            min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.9, 0.9),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,
        )
        visual_effect_generator = random_visual_effect_generator(
            contrast_range=(0.9, 1.1),
            brightness_range=(-.1, .1),
            hue_range=(-0.05, 0.05),
            saturation_range=(0.95, 1.05)
        )
    else:
        transform_generator = random_transform_generator(flip_x_chance=0.5)
        visual_effect_generator = None

    if args.dataset_type == 'coco':
        # import here to prevent unnecessary dependency on cocoapi
        from ..preprocessing.coco import CocoGenerator

        train_generator = CocoGenerator(
            args.coco_path,
            'train2017',
            transform_generator=transform_generator,
            visual_effect_generator=visual_effect_generator,
            **common_args
        )

        validation_generator = CocoGenerator(
            args.coco_path,
            'val2017',
            shuffle_groups=False,
            **common_args
        )
    elif args.dataset_type == 'pascal':
        train_generator = PascalVocGenerator(
            args.pascal_path,
            'train',
            image_extension=args.image_extension,
            transform_generator=transform_generator,
            visual_effect_generator=visual_effect_generator,
            **common_args
        )

        validation_generator = PascalVocGenerator(
            args.pascal_path,
            'val',
            image_extension=args.image_extension,
            shuffle_groups=False,
            **common_args
        )
    elif args.dataset_type == 'csv':
        train_generator = CSVGenerator(
            args.annotations,
            args.classes,
            transform_generator=transform_generator,
            visual_effect_generator=visual_effect_generator,
            **common_args
        )

        if args.val_annotations:
            validation_generator = CSVGenerator(
                args.val_annotations,
                args.classes,
                shuffle_groups=False,
                **common_args
            )
        else:
            validation_generator = None
    elif args.dataset_type == 'oid':
        train_generator = OpenImagesGenerator(
            args.main_dir,
            subset='train',
            version=args.version,
            labels_filter=args.labels_filter,
            annotation_cache_dir=args.annotation_cache_dir,
            parent_label=args.parent_label,
            transform_generator=transform_generator,
            visual_effect_generator=visual_effect_generator,
            **common_args
        )

        validation_generator = OpenImagesGenerator(
            args.main_dir,
            subset='validation',
            version=args.version,
            labels_filter=args.labels_filter,
            annotation_cache_dir=args.annotation_cache_dir,
            parent_label=args.parent_label,
            shuffle_groups=False,
            **common_args
        )
    elif args.dataset_type == 'kitti':
        train_generator = KittiGenerator(
            args.kitti_path,
            subset='train',
            transform_generator=transform_generator,
            visual_effect_generator=visual_effect_generator,
            **common_args
        )

        validation_generator = KittiGenerator(
            args.kitti_path,
            subset='val',
            shuffle_groups=False,
            **common_args
        )
    else:
        raise ValueError('Invalid data type received: {}'.format(args.dataset_type))

    return train_generator, validation_generator

train_gen, valid_gen = create_generators(args, b.preprocess_image)

In [ ]:

# # from keras_retinanet.bin.train import create_generators
# # from keras_retinanet.models import backbone
# from keras_retinanet.utils.config import read_config_file,parse_anchor_parameters
# # # from keras_retinanet.bin.debug import run
# from google.colab.patches import cv2_imshow
# from keras_retinanet.utils.anchors import anchors_for_shape, compute_gt_annotations
# from keras_retinanet.utils.visualization import draw_annotations, draw_boxes, draw_caption


# def run(generator, args, anchor_params):
#     """ Main loop.

#     Args
#         generator: The generator to debug.
#         args: parseargs args object.
#     """
#     # display images, one at a time
#     i = 0
#     while True:
#         # load the data
#         image       = generator.load_image(i)
#         annotations = generator.load_annotations(i)
#         if len(annotations['labels']) > 0 :
#             # apply random transformations
#             if args.random_transform:
#                 image, annotations = generator.random_transform_group_entry(image, annotations)
#                 image, annotations = generator.random_visual_effect_group_entry(image, annotations)

#             # resize the image and annotations
#             if args.resize:
#                 image, image_scale = generator.resize_image(image)
#                 annotations['bboxes'] *= image_scale

#             anchors = anchors_for_shape(image.shape, anchor_params=anchor_params)
#             positive_indices, _, max_indices = compute_gt_annotations(anchors, annotations['bboxes'])

#             # draw anchors on the image
#             if args.anchors:
#                 draw_boxes(image, anchors[positive_indices], (255, 255, 0), thickness=1)

#             # draw annotations on the image
#             if args.annotations:
#                 # draw annotations in red
#                 draw_annotations(image, annotations, color=(0, 0, 255), label_to_name=generator.label_to_name)

#                 # draw regressed anchors in green to override most red annotations
#                 # result is that annotations without anchors are red, with anchors are green
#                 draw_boxes(image, annotations['bboxes'][max_indices[positive_indices], :], (0, 255, 0))

#             # display name on the image
#             if args.display_name:
#                 draw_caption(image, [0, image.shape[0]], os.path.basename(generator.image_path(i)))

#         # write to file and advance if no-gui selected
#         if args.no_gui:
#             output_path = make_output_path(args.output_dir, generator.image_path(i), flatten=args.flatten_output)
#             os.makedirs(os.path.dirname(output_path), exist_ok=True)
#             cv2.imwrite(output_path, image)
#             i += 1
#             if i == generator.size():  # have written all images
#                 break
#             else:
#                 continue

#         # if we are using the GUI, then show an image
#         cv2_imshow(image)
#         key = cv2.waitKeyEx()

#         # press right for next image and left for previous (linux or windows, doesn't work for macOS)
#         # if you run macOS, press "n" or "m" (will also work on linux and windows)

#         if key in rightkeys:
#             i = (i + 1) % generator.size()
#         if key in leftkeys:
#             i -= 1
#             if i < 0:
#                 i = generator.size() - 1

#         # press q or Esc to quit
#         if (key == ord('q')) or (key == 27):
#             return False

#     return True


# # train_gen, valid_gen = create_generators(args,b.preprocess_image)
# while run(train_gen, args, parse_anchor_parameters(args.config)):
#     pass

In [ ]:
import imgaug as ia
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# Define our sequence of augmentation steps that will be applied to every image.
seq = iaa.Sequential(
[      
        # sometimes(iaa.Crop(percent=(0, 0.2))),
 
        #  sometimes(iaa.Affine(
        #     scale={"x": (0.9, 1.2), "y": (0.9, 1.2)},
        #     translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        #     rotate=(-45, 45),
        #     shear=(-16, 16),
        #     order=[0, 1],
        #     cval=(0, 255),
        #     mode=ia.ALL
        # )),

        #   # Change brightness of images (85-115% of original value).
        #   iaa.Multiply((0.90, 1.15), per_channel=0.5),

        #   # # Improve or worsen the contrast of images.
        #   iaa.ContrastNormalization((0.75, 1.25), per_channel=0.5),

          # Convert each image to grayscale and then overlay the
          # result with the original with random alpha. I.e. remove
          # colors with varying strengths.
          # iaa.Grayscale(alpha=(0.0, 0.25)),
#
        # Execute 1 to 9 of the following (less important) augmenters per
        # image. Don't execute all of them, as that would often be way too
        # strong.
        #
        iaa.SomeOf((1, 9),
            [

                        # Blur each image with varying strength using
                        # gaussian blur (sigma between 0 and .5),
                        # average/uniform blur (kernel size 1x1)
                        # median blur (kernel size 1x1).
                        iaa.OneOf([
                            iaa.GaussianBlur((0,1)),
                            iaa.AverageBlur(k=(2,2)),
                            iaa.MedianBlur(k=(1,1)),
                        ]),

                        # Sharpen each image, overlay the result with the original
                        # image using an alpha between 0 (no sharpening) and 1
                        # (full sharpening effect).
                        iaa.Sharpen(alpha=(0, 0.25), lightness=(0.75, 1.5)),

                        # Add gaussian noise to some images.
                        # In 50% of these cases, the noise is randomly sampled per
                        # channel and pixel.
                        # In the other 50% of all cases it is sampled once per
                        # pixel (i.e. brightness change).
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.01*255), per_channel=0.5
                        ),

                        # Either drop randomly 1 to 10% of all pixels (i.e. set
                        # them to black) or drop them on an image with 2-5% percent
                        # of the original size, leading to large dropped
                        # rectangles.
                        # iaa.OneOf([
                        #     iaa.Dropout((0.01, 0.1), per_channel=0.5),
                        #     iaa.CoarseDropout(
                        #         (0.03, 0.15), size_percent=(0.02, 0.05),
                        #         per_channel=0.2
                        #     ),
                        # ]),
             
                        
                        # Invert each image's channel with 5% probability.
                        # This sets each pixel value v to 255-v.
                        iaa.Invert(0.05, per_channel=True), # invert color channels

                        # Add a value of -5 to 5 to each pixel.
                        iaa.Add((-5, 5), per_channel=0.5),

                        # # Change brightness of images (85-115% of original value).
                        iaa.Multiply((0.85, 1.15), per_channel=0.5),

                        # # Improve or worsen the contrast of images.
                        iaa.ContrastNormalization((0.75, 1.25), per_channel=0.5),

                        # Convert each image to grayscale and then overlay the
                        # result with the original with random alpha. I.e. remove
                        # colors with varying strengths.
                        # iaa.Grayscale(alpha=(0.0, 0.25)),

                        # In some images distort local areas with varying strength.
                        sometimes(iaa.PiecewiseAffine(scale=(0.001, 0.01)))
                    ],
            # do all of the above augmentations in random order
            random_order=True
        )
    ],
    # do all of the above augmentations in random order
    random_order=True
)

def augment_train_gen(train_gen, visualize=False):
    '''
    Creates a generator using another generator with applied image augmentation.
    Args
        train_gen  : keras-retinanet generator object.
        visualize  : Boolean; False will convert bounding boxes to their anchor box targets for the model.
    '''
    imgs = []
    boxes = []
    targets = []
    size = train_gen.size()
    idx = 0
    while True:
        while len(imgs) < args.batch_size:
            image       = train_gen.load_image(idx % size)
            annotations = train_gen.load_annotations(idx % size)
            image,annotations = train_gen.random_transform_group_entry(image,annotations)
            imgs.append(image)            
            boxes.append(annotations['bboxes'])
            targets.append(annotations)
            idx += 1
        if visualize:
            imgs = seq.augment_images(imgs)
            imgs = np.array(imgs)
            boxes = np.array(boxes)
            yield imgs,boxes
        else:
            imgs = seq.augment_images(imgs)
            imgs,targets = train_gen.preprocess_group(imgs,targets)
            imgs = train_gen.compute_inputs(imgs)
            targets = train_gen.compute_targets(imgs,targets)
            imgs = np.array(imgs)
            yield imgs,targets
        imgs = []
        boxes = []
        targets = []
		
		
# import matplotlib.pyplot as plt

# skip_batches = 5
# i = 0
# for imgs,boxes in augment_train_gen(train_gen,visualize=True):
#     if i > skip_batches:
#         fig=plt.figure(figsize=(24,96))
#         columns = 2
#         rows = 8
#         for i in range(1, columns*rows + 1):
#             draw_boxes(imgs[i], boxes[i], (0, 255, 0), thickness=1)
#             fig.add_subplot(rows, columns, i)
#             plt.imshow(cv2.cvtColor(imgs[i],cv2.COLOR_BGR2RGB))
#         plt.show()
#         break
#     else:
#         i += 1

model, training_model, prediction_model = create_models(
            backbone_retinanet=b.retinanet,
            num_classes=train_gen.num_classes(),
            weights=None,
            multi_gpu=False,
            freeze_backbone=True,
            lr=1e-3,
            config=args.config
        )

tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32, numpy=
array([[-16.       , -16.       ,  16.       ,  16.       ],
       [-19.2      , -19.2      ,  19.2      ,  19.2      ],
       [-25.6      , -25.6      ,  25.6      ,  25.6      ],
       [-13.063946 , -19.595919 ,  13.063946 ,  19.595919 ],
       [-15.676735 , -23.515102 ,  15.676735 ,  23.515102 ],
       [-20.902313 , -31.35347  ,  20.902313 ,  31.35347  ],
       [-11.313708 , -22.627417 ,  11.313708 ,  22.627417 ],
       [-13.57645  , -27.1529   ,  13.57645  ,  27.1529   ],
       [-18.101934 , -36.20387  ,  18.101934 ,  36.20387  ],
       [-10.119288 , -25.298222 ,  10.119288 ,  25.298222 ],
       [-12.1431465, -30.357866 ,  12.1431465,  30.357866 ],
       [-16.190863 , -40.477154 ,  16.190863 ,  40.477154 ],
       [ -9.237604 , -27.712812 ,   9.237604 ,  27.712812 ],
       [-11.085126 , -33.25538  ,  11.085126 ,  33.25538  ],
       [-14.780168 , -44.3405   ,  14.780168 ,  44.3405   ]],
      dtype=f

# Retraining the foot model with better parameters
- config file  -> # Updating the parameters
# New ratios parameters for training as previous didn't help
with open('config.ini','w') as f:
    f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512 1024\nstrides = 8 16 32 64 128 256\nratios  = 1 1.5 2 2.5 3\nscales  = 1 1.2 1.6\n')
- iou 0.6 NMS 0.5 and score threshold 0.15

## Continued training from 21 epoch

In [ ]:
from keras_retinanet.callbacks import RedirectModel 
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    valid_gen,
    args,
)
training_model.load_weights('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2/resnet50_csv_21_class_loss_0.0973.h5', skip_mismatch = True, by_name = True)

training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

Epoch 1/200
44/44 [==============================] - 258s 6s/step - loss: 1.4052 - regression_loss: 1.2905 - classification_loss: 0.1146

Epoch 00001: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2 continued/resnet50_csv_01.h5
Epoch 2/200
44/44 [==============================] - 227s 5s/step - loss: 1.4634 - regression_loss: 1.3478 - classification_loss: 0.1156

Epoch 00002: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2 continued/resnet50_csv_02.h5
Epoch 3/200
44/44 [==============================] - 242s 5s/step - loss: 1.4614 - regression_loss: 1.3458 - classification_loss: 0.1157

Epoch 00003: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2 continued/resnet50_csv_03.h5
Epoch 4/200
44/44 [==============================] - 234s 5s/step - loss: 1.4859 - regression_loss: 1.3659 - classificati

In [ ]:
from keras_retinanet.callbacks import RedirectModel 
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    valid_gen,
    args,
)
training_model.load_weights('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_98_class_loss_0.2137.h5', skip_mismatch = True, by_name = True)

training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer regression_submodel due to mismatch in shape ((3, 3, 256, 60) vs (132, 256, 3, 3)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer regression_submodel due to mismatch in shape ((60,) vs (132,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 75) vs (165, 256, 3, 3)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((75,) vs (165,)).
  weight_values[i].shape))


Epoch 1/200
44/44 [==============================] - 281s 6s/step - loss: 2.2704 - regression_loss: 1.8941 - classification_loss: 0.3763

Epoch 00001: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2/resnet50_csv_01.h5
Epoch 2/200
44/44 [==============================] - 215s 5s/step - loss: 1.6377 - regression_loss: 1.4739 - classification_loss: 0.1638

Epoch 00002: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2/resnet50_csv_02.h5
Epoch 3/200
44/44 [==============================] - 225s 5s/step - loss: 1.5598 - regression_loss: 1.4174 - classification_loss: 0.1424

Epoch 00003: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/Model v2/resnet50_csv_03.h5
Epoch 4/200
44/44 [==============================] - 217s 5s/step - loss: 1.5352 - regression_loss: 1.4024 - classification_loss: 0.1329

Epoch 00004: 

# Previous version -> picked up class loss 0.2137 model

In [ ]:
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    valid_gen,
    args,
)
training_model.load_weights('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/resnet50_coco_best_v2.0.1.h5', skip_mismatch = True, by_name = True)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer regression_submodel due to mismatch in shape ((3, 3, 256, 132) vs (36, 256, 3, 3)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer regression_submodel due to mismatch in shape ((132,) vs (36,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 165) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((165,) vs (720,)).
  weight_values[i].shape))


## Will train a new model for Foot joints

# Training the final foot model with below parameters
- Iou = 0.5
- NMS = 0.5
- score threshold = 0.05
- Ratios = 0.8 1 1.3 1.5 1.8 2 2.2 2.5 2.8 3.0 3.2
- scales  = 1 1.2 1.6
- IMgae min size = 1000
- Image max side = 1400 
- Evaluation = False

In [ ]:
training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

Epoch 1/200
88/88 [==============================] - 270s 3s/step - loss: 2.9207 - regression_loss: 2.2971 - classification_loss: 0.6236

Epoch 00001: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_01.h5


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `mAP` which is not available. Available metrics are: loss,regression_loss,classification_loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
88/88 [==============================] - 263s 3s/step - loss: 2.5422 - regression_loss: 2.1210 - classification_loss: 0.4212

Epoch 00002: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_02.h5
Epoch 3/200
88/88 [==============================] - 285s 3s/step - loss: 2.4291 - regression_loss: 2.0359 - classification_loss: 0.3932

Epoch 00003: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_03.h5
Epoch 4/200
88/88 [==============================] - 264s 3s/step - loss: 2.3440 - regression_loss: 1.9798 - classification_loss: 0.3642

Epoch 00004: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_04.h5
Epoch 5/200
88/88 [==============================] - 278s 3s/step - loss: 2.3815 - regression_loss: 2.0251 - classification_loss: 0.3564

Epoch 00005: saving model to /content/dr

KeyboardInterrupt: ignored

## Visualizing anchor boxes

In [ ]:

from keras_retinanet.bin.train import create_generators
from keras_retinanet.models import backbone
from keras_retinanet.utils.config import read_config_file,parse_anchor_parameters
from keras_retinanet.bin.debug import run


b = backbone('resnet50')

class args:
    batch_size = 4
    config = read_config_file('config.ini')
    random_transform = True # Image augmentation
    annotations = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/all_hand_train_03_06.csv'
    val_annotations = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/all_hand_val_03_06.csv'
    classes = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/retinanet train val files new/Hands class names.csv'
    image_min_side = 1000
    image_max_side = 1400
    no_resize = False
    dataset_type = 'csv'
    tensorboard_dir = ''
    evaluation = False
    snapshots = True
    snapshot_path = "/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Hand models New annotations"
    backbone = 'resnet50'
    epochs = 200
    steps = len(train_ids)//(batch_size)
    weighted_average = True
    reduce_lr_factor = 0.1
    reduce_lr_patience = 4
    compute_val_loss = True
    iou_threshold = 0.5
    nms_threshold = 0.5
    score_threshold = 0.05

train_gen, valid_gen = create_generators(args,b.preprocess_image)
while run(valid_gen,args,parse_anchor_parameters(args.config)):
    pass

## Retraining from epoch 16 Freeze previousm model
-- mAP -> 0.40 class loss 0.41

In [ ]:
training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

Epoch 1/150
88/88 [==============================] - 276s 3s/step - loss: 2.7548 - regression_loss: 2.3195 - classification_loss: 0.4353


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:29 Time:  0:00:29
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.3656
60 instances of class fin_1 with average precision: 0.3669
60 instances of class fin_2 with average precision: 0.3945
60 instances of class fin_3 with average precision: 0.2406
61 instances of class fin_4 with average precision: 0.4197
mAP: 0.3577

Epoch 00001: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_01.h5
Epoch 2/150
88/88 [==============================] - 223s 3s/step - loss: 2.7775 - regression_loss: 2.3440 - classification_loss: 0.4335


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:12 Time:  0:00:12
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.2231
60 instances of class fin_1 with average precision: 0.3517
60 instances of class fin_2 with average precision: 0.4389
60 instances of class fin_3 with average precision: 0.2556
61 instances of class fin_4 with average precision: 0.6506
mAP: 0.3848

Epoch 00002: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_02.h5
Epoch 3/150
88/88 [==============================] - 229s 3s/step - loss: 2.7782 - regression_loss: 2.3422 - classification_loss: 0.4360


...

60 instances of class fin_ip with average precision: 0.5531
60 instances of class fin_1 with average precision: 0.4473
60 instances of class fin_2 with average precision: 0.3587
60 instances of class fin_3 with average precision: 0.3921
61 instances of class fin_4 with average precision: 0.8056
mAP: 0.5123

Epoch 00013: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_13.h5

Epoch 00013: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 14/150
88/88 [==============================] - 246s 3s/step - loss: 2.6953 - regression_loss: 2.2876 - classification_loss: 0.4077


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:13 Time:  0:00:13
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.6430
60 instances of class fin_1 with average precision: 0.4377
60 instances of class fin_2 with average precision: 0.3704
60 instances of class fin_3 with average precision: 0.3344
61 instances of class fin_4 with average precision: 0.8179
mAP: 0.5217

Epoch 00014: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_14.h5
Epoch 15/150
88/88 [==============================] - 241s 3s/step - loss: 2.7134 - regression_loss: 2.3005 - classification_loss: 0.4130


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:13 Time:  0:00:13
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.6200
60 instances of class fin_1 with average precision: 0.4274
60 instances of class fin_2 with average precision: 0.3576
60 instances of class fin_3 with average precision: 0.3356
61 instances of class fin_4 with average precision: 0.8076
mAP: 0.5106

Epoch 00015: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_15.h5


## Have reduced the ratios in this run and changed the anchors sizes

In [ ]:
training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

Epoch 1/150
88/88 [==============================] - 299s 3s/step - loss: 3.0064 - regression_loss: 2.4081 - classification_loss: 0.5983


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.0623
60 instances of class fin_1 with average precision: 0.2203
60 instances of class fin_2 with average precision: 0.3422
60 instances of class fin_3 with average precision: 0.1276
61 instances of class fin_4 with average precision: 0.1462
mAP: 0.1796

Epoch 00001: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_01.h5
Epoch 2/150
88/88 [==============================] - 224s 3s/step - loss: 2.9202 - regression_loss: 2.4307 - classification_loss: 0.4895


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:12 Time:  0:00:12
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.1000
60 instances of class fin_1 with average precision: 0.2944
60 instances of class fin_2 with average precision: 0.3563
60 instances of class fin_3 with average precision: 0.1577
61 instances of class fin_4 with average precision: 0.1751
mAP: 0.2166

Epoch 00002: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_02.h5
Epoch 3/150
88/88 [==============================] - 231s 3s/step - loss: 2.9192 - regression_loss: 2.4162 - classification_loss: 0.5030


...

60 instances of class fin_ip with average precision: 0.1816
60 instances of class fin_1 with average precision: 0.2788
60 instances of class fin_2 with average precision: 0.4365
60 instances of class fin_3 with average precision: 0.1774
61 instances of class fin_4 with average precision: 0.7414
mAP: 0.3644

Epoch 00019: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_19.h5
Epoch 20/150
88/88 [==============================] - 231s 3s/step - loss: 2.7165 - regression_loss: 2.3052 - classification_loss: 0.4114


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:13 Time:  0:00:13
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.1949
60 instances of class fin_1 with average precision: 0.3115
60 instances of class fin_2 with average precision: 0.4527
60 instances of class fin_3 with average precision: 0.1827
61 instances of class fin_4 with average precision: 0.7172
mAP: 0.3730

Epoch 00020: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_20.h5
Epoch 21/150
88/88 [==============================] - 232s 3s/step - loss: 2.7139 - regression_loss: 2.3040 - classification_loss: 0.4098


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:13 Time:  0:00:13
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.1939
60 instances of class fin_1 with average precision: 0.2819
60 instances of class fin_2 with average precision: 0.4331
60 instances of class fin_3 with average precision: 0.1656
61 instances of class fin_4 with average precision: 0.7158
mAP: 0.3592

Epoch 00021: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_21.h5



# Logs for few changes made
- Ratios changed
- IOU threshold, NMS threshold = 0.6 and score threshold = 0.1

In [ ]:
training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

Epoch 1/150
88/88 [==============================] - 558s 6s/step - loss: 6.1760 - regression_loss: 2.4610 - classification_loss: 3.7149


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:01:13 Time:  0:01:13
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.0011
60 instances of class fin_1 with average precision: 0.0066
60 instances of class fin_2 with average precision: 0.0004
60 instances of class fin_3 with average precision: 0.0000
61 instances of class fin_4 with average precision: 0.0000
mAP: 0.0016

Epoch 00001: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_01.h5
Epoch 2/150
88/88 [==============================] - 346s 4s/step - loss: 4.7206 - regression_loss: 2.4290 - classification_loss: 2.2915


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:26 Time:  0:00:26
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.0000
60 instances of class fin_1 with average precision: 0.0000
60 instances of class fin_2 with average precision: 0.0000
60 instances of class fin_3 with average precision: 0.0023
61 instances of class fin_4 with average precision: 0.0059
mAP: 0.0017

Epoch 00002: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_02.h5
Epoch 3/150
88/88 [==============================] - 326s 4s/step - loss: 3.2642 - regression_loss: 2.4303 - classification_loss: 0.8339


...

60 instances of class fin_ip with average precision: 0.0000
60 instances of class fin_1 with average precision: 0.0000
60 instances of class fin_2 with average precision: 0.0000
60 instances of class fin_3 with average precision: 0.0012
61 instances of class fin_4 with average precision: 0.0000
mAP: 0.0002

Epoch 00011: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_11.h5

Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 12/150
88/88 [==============================] - 317s 4s/step - loss: 2.9970 - regression_loss: 2.4027 - classification_loss: 0.5943


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:27 Time:  0:00:27
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.0425
60 instances of class fin_1 with average precision: 0.0530
60 instances of class fin_2 with average precision: 0.0521
60 instances of class fin_3 with average precision: 0.0099
61 instances of class fin_4 with average precision: 0.0162
mAP: 0.0347

Epoch 00012: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_12.h5
Epoch 13/150
88/88 [==============================] - 317s 4s/step - loss: 2.8637 - regression_loss: 2.3463 - classification_loss: 0.5174


Running network: 100% (61 of 61) |#######| Elapsed Time: 0:00:27 Time:  0:00:27
Parsing annotations: 100% (61 of 61) |###| Elapsed Time: 0:00:00 Time:  0:00:00


60 instances of class fin_ip with average precision: 0.0315
60 instances of class fin_1 with average precision: 0.0831
60 instances of class fin_2 with average precision: 0.0748
60 instances of class fin_3 with average precision: 0.0182
61 instances of class fin_4 with average precision: 0.0204
mAP: 0.0455

Epoch 00013: saving model to /content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/Foot model with new annotations/resnet50_csv_13.h5


# Right foot joint detection

In [ ]:

# tf.set_random_seed(31) # SEEDS MAKE RESULTS MORE REPRODUCABLE
np.random.seed(17)
classes = np.arange(0, 11, 1).tolist()

# def convert_annotation(image_id,filename):
#     in_file = open('training_data/labels/%s.xml'%(image_id))
#     out_file = open(filename, 'a')
#     tree=ET.parse(in_file)
#     root = tree.getroot()
    
#     if root.iter('object') is not None:
#         for obj in root.iter('object'):
#             cls = obj.find('name').text
#             if cls not in classes:
#                 continue
#             cls_id = classes.index(cls)
            
#             xmlbox = obj.find('bndbox')
#             x1 = math.ceil(float(xmlbox.find('xmin').text))
#             y1 = math.ceil(float(xmlbox.find('ymin').text))
#             x2 = math.ceil(float(xmlbox.find('xmax').text))
#             y2 = math.ceil(float(xmlbox.find('ymax').text))
#             if x1 == x2 or y1 == y2:
#                 continue
                
#             out_file.write(f'training_data/images/{image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
#     else:
#         out_file.write(f'training_data/images/{image_id}.jpg,,,,,\n')


# _,_,image_ids = next(walk('training_data/images'))
# image_ids = [i[:-4] for i in image_ids]
# open('annotations.csv','w')
# open('val_annotations.csv','w')

# train_ids,val_ids = train_test_split(image_ids,random_state=31,test_size=0)

# for image_id in train_ids:
#     convert_annotation(image_id,'annotations.csv')
        
# for image_id in val_ids:
#     convert_annotation(image_id,'val_annotations.csv')
    
# print(len(train_ids),len(val_ids))

import pandas as pd
tr_annots = pd.read_csv('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/Right foot train.csv', header = None)
tr_annots['ids'] = tr_annots[0].apply(lambda x: x.split("/")[-1].split(".")[0])
train_ids = tr_annots['ids'].unique().tolist()

import pandas as pd
tr_annots = pd.read_csv('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/Right foot val.csv', header = None)
tr_annots['ids'] = tr_annots[0].apply(lambda x: x.split("/")[-1].split(".")[0])
val_ids = tr_annots['ids'].unique().tolist()

print(len(train_ids), len(val_ids))

with open('config.ini','w') as f:
    f.write('[anchor_parameters]\nsizes   = 32 64 128 256 512\nstrides = 8 16 32 64 128\nratios  = 1 1.2 1.5 2 \nscales  = 2 3 4 \n')
	
b = backbone('resnet50')

class args:
    batch_size = 2
    config = read_config_file('config.ini')
    random_transform = True # Image augmentation
    annotations = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/Right foot train.csv'
    val_annotations = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/training_data/Right foot val.csv'
    classes = '/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/foot_class_names.csv'
    image_min_side = 900
    image_max_side = 1200
    no_resize = False
    dataset_type = 'csv'
    tensorboard_dir = ''
    evaluation = False
    snapshots = True
    snapshot_path = "/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/save right foot model/"
    backbone = 'resnet50'
    epochs = 100
    steps = len(train_ids)//(batch_size)
    weighted_average = True
	
train_gen, valid_gen = create_generators(args, b.preprocess_image)


sometimes = lambda aug: iaa.Sometimes(0.5, aug)
# Define our sequence of augmentation steps that will be applied to every image.
seq = iaa.Sequential(
    [
        #
        # Execute 1 to 9 of the following (less important) augmenters per
        # image. Don't execute all of them, as that would often be way too
        # strong.
        #
        iaa.SomeOf((1, 9),
            [

                        # Blur each image with varying strength using
                        # gaussian blur (sigma between 0 and .5),
                        # average/uniform blur (kernel size 1x1)
                        # median blur (kernel size 1x1).
                        iaa.OneOf([
                            iaa.GaussianBlur((0,0.5)),
                            iaa.AverageBlur(k=(1)),
                            iaa.MedianBlur(k=(1)),
                        ]),

                        # Sharpen each image, overlay the result with the original
                        # image using an alpha between 0 (no sharpening) and 1
                        # (full sharpening effect).
                        iaa.Sharpen(alpha=(0, 0.25), lightness=(0.75, 1.5)),

                        # Add gaussian noise to some images.
                        # In 50% of these cases, the noise is randomly sampled per
                        # channel and pixel.
                        # In the other 50% of all cases it is sampled once per
                        # pixel (i.e. brightness change).
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.01*255), per_channel=0.5
                        ),

                        # Either drop randomly 1 to 10% of all pixels (i.e. set
                        # them to black) or drop them on an image with 2-5% percent
                        # of the original size, leading to large dropped
                        # rectangles.
                        iaa.OneOf([
                            iaa.Dropout((0.01, 0.1), per_channel=0.5),
                            iaa.CoarseDropout(
                                (0.03, 0.15), size_percent=(0.02, 0.05),
                                per_channel=0.2
                            ),
                        ]),

                        # Add a value of -5 to 5 to each pixel.
                        iaa.Add((-5, 5), per_channel=0.5),

                        # Change brightness of images (85-115% of original value).
                        iaa.Multiply((0.85, 1.15), per_channel=0.5),

                        # Improve or worsen the contrast of images.
                        iaa.ContrastNormalization((0.75, 1.25), per_channel=0.5),

                        # Convert each image to grayscale and then overlay the
                        # result with the original with random alpha. I.e. remove
                        # colors with varying strengths.
                        iaa.Grayscale(alpha=(0.0, 0.25)),

                        # In some images distort local areas with varying strength.
                        sometimes(iaa.PiecewiseAffine(scale=(0.001, 0.01)))
                    ],
            # do all of the above augmentations in random order
            random_order=True
        )
    ],
    # do all of the above augmentations in random order
    random_order=True
)


def augment_train_gen(train_gen, visualize=False):
    '''
    Creates a generator using another generator with applied image augmentation.
    Args
        train_gen  : keras-retinanet generator object.
        visualize  : Boolean; False will convert bounding boxes to their anchor box targets for the model.
    '''
    imgs = []
    boxes = []
    targets = []
    size = train_gen.size()
    idx = 0
    while True:
        while len(imgs) < args.batch_size:
            image       = train_gen.load_image(idx % size)
            annotations = train_gen.load_annotations(idx % size)
            image,annotations = train_gen.random_transform_group_entry(image,annotations)
            imgs.append(image)            
            boxes.append(annotations['bboxes'])
            targets.append(annotations)
            idx += 1
        if visualize:
            imgs = seq.augment_images(imgs)
            imgs = np.array(imgs)
            boxes = np.array(boxes)
            yield imgs,boxes
        else:
            imgs = seq.augment_images(imgs)
            imgs,targets = train_gen.preprocess_group(imgs,targets)
            imgs = train_gen.compute_inputs(imgs)
            targets = train_gen.compute_targets(imgs,targets)
            imgs = np.array(imgs)
            yield imgs,targets
        imgs = []
        boxes = []
        targets = []
		
		
# import matplotlib.pyplot as plt

# skip_batches = 5
# i = 0
# for imgs,boxes in augment_train_gen(train_gen,visualize=True):
#     if i > skip_batches:
#         fig=plt.figure(figsize=(24,96))
#         columns = 2
#         rows = 8
#         for i in range(1, columns*rows + 1):
#             draw_boxes(imgs[i], boxes[i], (0, 255, 0), thickness=1)
#             fig.add_subplot(rows, columns, i)
#             plt.imshow(cv2.cvtColor(imgs[i],cv2.COLOR_BGR2RGB))
#         plt.show()
#         break
#     else:
#         i += 1

model, training_model, prediction_model = create_models(
            backbone_retinanet=b.retinanet,
            num_classes=train_gen.num_classes(),
            weights=None,
            multi_gpu=False,
            freeze_backbone=True,
            lr=1e-3,
            config=args.config
        )

		
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    valid_gen,
    args,
)
training_model.load_weights('/content/drive/My Drive/RA2/Retinanet/Upload for Kaggle/resnet50_csv_27.h5', skip_mismatch = True, by_name = True)

##m Final model for Right hand joints detections
training_model.fit_generator(generator=augment_train_gen(train_gen),
        steps_per_epoch = args.steps,
        epochs=args.epochs,
        verbose=1,
        callbacks=callbacks, ) 

14/34 [===========>..................] - ETA: 32s - loss: 1.1881 - regression_loss: 1.0194 - classification_loss: 0.1686

KeyboardInterrupt: ignored